<a href="https://colab.research.google.com/github/tayebsami/emulator/blob/master/TPE_RECHARGE.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [8]:
import pandas as pd
import numpy as np
from google.colab import files

# ============================================================
# 1. CHARGEMENT DU FICHIER
# ============================================================

print("Veuillez sélectionner votre fichier Excel...")
uploaded = files.upload()

filename = list(uploaded.keys())[0]
print(f"Fichier chargé : {filename}\n")

df = pd.read_excel(filename)
print(f"Dimensions initiales : {df.shape[0]} lignes × {df.shape[1]} colonnes\n")
print("Aperçu des données :")
display(df.head())
print("="*80)

# ============================================================
# 2. PARAMÈTRES DE NETTOYAGE
# ============================================================

colonnes_a_supprimer = [
    'cl Client Image', 'Code', 'Référence', 'Famille',
    'CA HT Facturée'        # On NE supprime PLUS "CA HT Livrée"
]

codes_a_supprimer = ['1901','1902','1903','1904','1905','1906','1907']

# Ligne dupliquée exacte à supprimer
ligne_dupliquee = {
    'cl Client Image': 'cl Client Image',
    'Code': 'Code',
    'Référence': 'Référence',
    'Client': 'Client',
    'Famille': 'Famille',
    'CA HT Livrée': 'CA HT Livrée',
    'CA HT Facturée': 'CA HT Facturée'
}

# ============================================================
# 3. FONCTIONS DE NETTOYAGE
# ============================================================

def nettoyer_ca_ht(df):
    """Nettoyer et convertir CA HT Livrée (en priorité)."""
    if 'CA HT Livrée' not in df.columns:
        print("Priorité CA HT Livrée : colonne absente.")
        return df

    df['CA HT Livrée'] = (
        df['CA HT Livrée']
        .astype(str)
        .str.replace(' ', '', regex=False)
        .str.replace(',', '', regex=False)
    )

    df['CA HT Livrée'] = pd.to_numeric(df['CA HT Livrée'], errors='coerce')

    print("Priorité : Nettoyage de la colonne 'CA HT Livrée' effectué.")
    return df


def supprimer_client_vide(df):
    """Supprimer les lignes où Client = 'Client' et CA HT Livrée est vide."""
    if 'Client' not in df.columns or 'CA HT Livrée' not in df.columns:
        print("Étape Client vide : colonnes manquantes.")
        return df

    lignes_avant = df.shape[0]

    condition = (
        df['Client'].astype(str).str.strip() == "Client"
    ) & (
        df['CA HT Livrée'].isna()
    )

    df = df[~condition]
    suppr = lignes_avant - df.shape[0]

    print(f"Étape Client vide : {suppr} ligne(s) supprimée(s).")
    return df


def supprimer_ligne_dupliquee(df, ligne):
    """Supprime la ligne d’en-tête dupliquée au milieu des données."""
    filtres = [df[col].astype(str) == str(val) for col, val in ligne.items() if col in df.columns]

    if not filtres:
        print("Étape ligne dupliquée : aucune colonne correspondante.")
        return df

    condition = filtres[0]
    for f in filtres[1:]:
        condition &= f

    lignes_avant = df.shape[0]
    df = df[~condition]
    suppr = lignes_avant - df.shape[0]

    print(f"Étape ligne dupliquée : {suppr} ligne(s) supprimée(s).")
    return df


def supprimer_colonnes(df, colonnes):
    existantes = [col for col in colonnes if col in df.columns]
    df = df.drop(columns=existantes)
    print(f"Étape colonnes : {len(existantes)} colonnes supprimées : {existantes}")
    return df


def supprimer_clients_code(df, filename, codes):
    original_length = df.shape[0]
    df_temp = pd.read_excel(filename)

    if "Code" not in df_temp.columns:
        print("Étape codes : colonne 'Code' absente.")
        return df

    indices = df_temp[df_temp['Code'].astype(str).isin(codes)].index
    df = df.drop(index=indices, errors='ignore')

    removed = original_length - df.shape[0]
    print(f"Étape codes : {removed} lignes supprimées pour codes {codes}.")
    return df


def supprimer_lignes_vides(df):
    lignes_avant = df.shape[0]
    df = df.dropna(how='all')
    print(f"Étape vides : {lignes_avant - df.shape[0]} lignes vides supprimées.")
    return df


# ============================================================
# 4. PIPELINE DE NETTOYAGE (ORDRE OPTIMAL)
# ============================================================

print("\n=== DÉBUT DU NETTOYAGE ===\n")

df = nettoyer_ca_ht(df)                            # 1) PRIORITÉ : CA HT Livrée
df = supprimer_client_vide(df)                     # 2) Supprimer les lignes "Client" vides
df = supprimer_ligne_dupliquee(df, ligne_dupliquee)# 3) En-tête dupliqué
df = supprimer_colonnes(df, colonnes_a_supprimer)  # 4) Colonnes inutiles
df = supprimer_clients_code(df, filename, codes_a_supprimer)  # 5) Codes blacklistés
df = supprimer_lignes_vides(df)                    # 6) Lignes vides

print("\n=== NETTOYAGE TERMINÉ ===")
print(f"Dimensions finales : {df.shape[0]} lignes × {df.shape[1]} colonnes")
display(df.head(15))
print("="*80)

# ============================================================
# 5. EXPORT
# ============================================================

output_filename = filename.rsplit('.', 1)[0] + '_nettoyé.xlsx'
df.to_excel(output_filename, index=False)

print(f"Fichier nettoyé sauvegardé : {output_filename}")
files.download(output_filename)
print("\nProcessus terminé.")


Veuillez sélectionner votre fichier Excel...


Saving TPE-RECHARGE.xlsx to TPE-RECHARGE (4).xlsx
Fichier chargé : TPE-RECHARGE (4).xlsx

Dimensions initiales : 43 lignes × 8 colonnes

Aperçu des données :


,cl Client Image,Code,Référence,Client,Famille,Qté livrée,CA HT Livrée,CA HT Facturée
0,NaN,13-COD/002DCS,AYEN FORCE TELECOM,ALI BELHADJ (ABDELOUAHAB) AYEN FORCE,Clients gros,10000,990000,0
1,NaN,13-COD/015DCS,NaN,BENGRINE ALI (ALI TELECOM),Clients gros,2000,198000,0
2,NaN,35-COD/007,NaN,BOURCHOUCHE MORAD,Clients gros,10000,985000,0
3,NaN,16-COD/043DCS,NaN,BOUZEFFOUR FAYCAL,Clients gros,15000,1477500,0
4,NaN,09-COD/020DCS,NaN,CHERRATI REDHOUANE,Clients gros,1000,99000,0



=== DÉBUT DU NETTOYAGE ===

Priorité : Nettoyage de la colonne 'CA HT Livrée' effectué.
Étape Client vide : 0 ligne(s) supprimée(s).
Étape ligne dupliquée : 0 ligne(s) supprimée(s).
Étape colonnes : 5 colonnes supprimées : ['cl Client Image', 'Code', 'Référence', 'Famille', 'CA HT Facturée']
Étape codes : 0 lignes supprimées pour codes ['1901', '1902', '1903', '1904', '1905', '1906', '1907'].
Étape vides : 0 lignes vides supprimées.

=== NETTOYAGE TERMINÉ ===
Dimensions finales : 43 lignes × 3 colonnes


,Client,Qté livrée,CA HT Livrée
0,ALI BELHADJ (ABDELOUAHAB) AYEN FORCE,10000,990000
1,BENGRINE ALI (ALI TELECOM),2000,198000
2,BOURCHOUCHE MORAD,10000,985000
3,BOUZEFFOUR FAYCAL,15000,1477500
4,CHERRATI REDHOUANE,1000,99000
5,DJELLOUL KOUADRI HAMAD,2500,241250
6,EL HOUARI LAKEHAL (HOUARI LEGAL) LH TELECOM,15000,1477500
7,GACEM ABDELMADJID,15000,1477500
8,GSM SELMA MOHAMED EL AMINE,15000,1477500
9,KERBICHE MAAMAR ( MAAMAR MOSTA ),15000,1485000


Fichier nettoyé sauvegardé : TPE-RECHARGE (4)_nettoyé.xlsx


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>


Processus terminé.
